<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mistral_Integration_with_Langchain_PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
#Install Libraries to access Google Drive and OpenAI resources.
%pip install colab-env --upgrade --quiet --root-user-action=ignore
import colab_env

  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


# Definitions

In [2]:
def insert_document(id,embedding):
    #review_embedding=get_embedding(text)
    ### INSERT INTO DB
    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    conn = ps.connect(database=DB_NAME,
							user=DB_USER,
							password=DB_PASS,
							host=DB_HOST,
							port=DB_PORT)
    cur = conn.cursor() # creating a cursor
    cur.execute("""
        INSERT INTO documents
        (id, embedding)
        VALUES ('%s',
                '%s')""" % (id,embedding))

    conn.commit()
    print("INSERT EMBEDDING %s successfully"%embedding)
    conn.close()
    cur.close()

In [3]:
def query_selection(query):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""
          %s """%query)
      records = cur.fetchall()
      print("Total rows are:  ", len(records))
      print("Printing each row")
      print()
      n=0
      for row in records:
          n=n+1
          print("ROW %s: "%n, row)
      conn.close()
      cur.close()
      print()
      print("QUERY SELECTION successfully")
      print()

In [4]:
def drop_extension(extension):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""DROP EXTENSION IF EXISTS %s CASCADE"""%extension)
      cur.query
      conn.commit()
      cur.close()
      conn.close()

      print()
      print("DROP EXTENSION %s successfully"%extension)
      print()

# PostgreSQL: Definition and Configuration

In [5]:
# install PSQL WITH DEV Libraries AND PG embedding
!apt install postgresql postgresql-contrib &>log
!service postgresql restart
!sudo apt install postgresql-server-dev-all

 * Restarting PostgreSQL 14 database server
   ...done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libffi-dev libpfm4 libz3-4 libz3-dev llvm-14 llvm-14-dev
  llvm-14-runtime llvm-14-tools postgresql-server-dev-14 python3-pygments
  python3-yaml
Suggested packages:
  llvm-14-doc python-pygments-doc ttf-bitstream-vera
The following NEW packages will be installed:
  binfmt-support libffi-dev libpfm4 libz3-4 libz3-dev llvm-14 llvm-14-dev
  llvm-14-runtime llvm-14-tools postgresql-server-dev-14
  postgresql-server-dev-all python3-pygments python3-yaml
0 upgraded, 13 newly installed, 0 to remove and 24 not upgraded.
Need to get 59.8 MB of archives.
After this operation, 361 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 python3-yaml amd64 5.4.1-1ubuntu1 [129 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 bi

In [6]:
!cp -pr /content/gdrive/MyDrive/tools/pg_embedding /content/
%cd /content/pg_embedding/
print()
print('START: PG embedding COMPILATION')
!make
!make install # may need sudo
#!make uninstall
print('END: PG embedding COMPILATION')
print()

/content/pg_embedding

START: PG embedding COMPILATION
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -fno-omit-frame-pointer -Ofast -fPIC -I. -I./ -I/usr/include/postgresql/14/server -I/usr/include/postgresql/internal  -Wdate-time -D_FORTIFY_SOURCE=2 -D_GNU_SOURCE -I/usr/include/libxml2   -c -o embedding.o embedding.c
g++ -Wall -Wpointer-arith -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -std=c++11 -f

In [7]:
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")

/content
ALTER ROLE

Extensions Available:
Total rows are:   47
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('se

In [8]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#!sudo -u postgres psql -c "DROP EXTENSION vector CASCADE"
#!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding real[])"

h="{1,2,3}"
hh= "INSERT INTO documents(id, embedding) VALUES (1,'%s'), (2,'{4,5,6}')"%h
print(hh)

#del insert_document
insert_document(1,'{1,2,3}')
insert_document(2,'{4,5,6}')

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5)"

/content
ALTER ROLE
CREATE EXTENSION
ERROR:  table "documents" does not exist
CREATE TABLE
INSERT INTO documents(id, embedding) VALUES (1,'{1,2,3}'), (2,'{4,5,6}')
INSERT EMBEDDING {1,2,3} successfully
INSERT EMBEDDING {4,5,6} successfully


In [9]:
!cp -pr /content/gdrive/MyDrive/tools/pgvector /content/
%cd /content/pgvector/
print()
print('START: PG VECTOR COMPILATION')
!make
!make install
#!make uninstall
print('END: PG VECTOR COMPILATION')
print()

/content/pgvector

START: PG VECTOR COMPILATION
make: Nothing to be done for 'all'.
/bin/mkdir -p '/usr/lib/postgresql/14/lib'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/usr/bin/install -c -m 755  vector.so '/usr/lib/postgresql/14/lib/vector.so'
/usr/bin/install -c -m 644 .//vector.control '/usr/share/postgresql/14/extension/'
/usr/bin/install -c -m 644 .//sql/vector--0.1.0--0.1.1.sql .//sql/vector--0.1.1--0.1.3.sql .//sql/vector--0.1.3--0.1.4.sql .//sql/vector--0.1.4--0.1.5.sql .//sql/vector--0.1.5--0.1.6.sql .//sql/vector--0.1.6--0.1.7.sql .//sql/vector--0.1.7--0.1.8.sql .//sql/vector--0.1.8--0.2.0.sql .//sql/vector--0.2.0--0.2.1.sql .//sql/vector--0.2.1--0.2.2.sql .//sql/vector--0.2.2--0.2.3.sql .//sql/vector--0.2.3--0.2.4.sql .//sql/vector--0.2.4--0.2.5.sql .//sql/vector--0.2.5--0.2.6.sql .//sql/vector--0.2.6--0.2.7.sql .//sql/vector--0.2.7--0.3.0.sql .//sql/vector--0.3.0--0.3.1.sql .//sql/vector--0.3.1--0.3.2.sql .//sql/v

In [10]:
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"
print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")


Extensions Available:
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('ssli

In [11]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"
!sudo -u postgres psql -c "CREATE EXTENSION vector"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding vector(3))"

insert_document(1,'[1,2,3]')
insert_document(2,'[4,5,6]')

!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw (embedding vector_l2_ops)"

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (m=3)"


/content
ALTER ROLE
DROP EXTENSION
CREATE EXTENSION
DROP TABLE
CREATE TABLE
INSERT EMBEDDING [1,2,3] successfully
INSERT EMBEDDING [4,5,6] successfully
CREATE INDEX


In [12]:
## <-> and <=> Euclidean (L2) and Cosine distances NO Manhattan distance: <~>
query_selection("SELECT * FROM documents ORDER BY embedding::vector <-> '[3,1,2]' LIMIT 5")

Total rows are:   2
Printing each row

ROW 1:  (1, '[1,2,3]')
ROW 2:  (2, '[4,5,6]')

QUERY SELECTION successfully



In [13]:
print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")


Extensions Available:
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('ssli

In [14]:
# Euclidean (L2) distance index:
#CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <-> array[3,3,3] LIMIT 1;

# Cosine distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <=> array[3,3,3] LIMIT 1;

# Manhattan distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_manhattan_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <~> array[3,3,3] LIMIT 1;

# AWS - Document Upload

In [15]:
#https://platform.openai.com/docs/guides/text-generation
%pip install openai==0.28  --root-user-action=ignore
%pip install "unstructured[all-docs]"
!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet
!pip install pypdf
%pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 69.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB

In [16]:
#!rm -rf /content/*.pdf
!mkdir -p /content/data/
%cd /content/data/

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "/content/data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    #print(file_path)
    urlretrieve(url, file_path)

/content/data


In [17]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

In [18]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

#%cd /content/data

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs)}')

# of Document Pages 25
# of Document Chunks: 299


# Langchain and LLM Configurations

In [19]:
#%pip install colab-env --upgrade --quiet --root-user-action=ignore
#!pip install accelerate

import torch
from textwrap import fill
from IPython.display import Markdown, display

import colab_env
import os

access_token = os.getenv("HF_TOKEN")

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

#from transformers import AutoTokenizer, MistralForCausalLM

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [20]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME='mistralai/Mistral-7B-Instruct-v0.1'
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.9
generation_config.top_p = 0.9
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

#model.to(device)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id
)

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

# Running Vector Queries

In [22]:
import openai

import os
collection_name='AWS'

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

%pip install colab-env
import colab_env

connection_string = os.getenv("DATABASE_URL")

# https://supabase.com/blog/fewer-dimensions-are-better-pgvector
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

%cd /content/

# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS vector CASCADE"
!sudo -u postgres psql -c "CREATE EXTENSION embedding"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding real[])"

h="{1,2,3}"
hh= "INSERT INTO documents(id, embedding) VALUES (1,'%s'), (2,'{4,5,6}')"%h
print(hh)

#del insert_document
insert_document(1,'{1,2,3}')
insert_document(2,'{4,5,6}')

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5)"

/content
/content
NOTICE:  drop cascades to column embedding of table documents
DROP EXTENSION
CREATE EXTENSION
DROP TABLE
CREATE TABLE
INSERT INTO documents(id, embedding) VALUES (1,'{1,2,3}'), (2,'{4,5,6}')
INSERT EMBEDDING {1,2,3} successfully
INSERT EMBEDDING {4,5,6} successfully


In [23]:
print()
#del embeddings
print("Extension available")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print()
print("Extension used")
query_selection("SELECT * FROM pg_extension order by 1")
print()


Extension available
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('sslinf

In [24]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)

query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)

#query = db.embedding_function.embed_query(query)
results_with_scores = db.similarity_search_with_score(query)

for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

print()
print("-" * 80)


--------------------------------------------------------------------------------
How has AWS evolved?
Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.52015233


Content: in AWS. Our new customer pipeline is robust, as are our active migrations. Many companies usediscontinuous periods like this to step back and determine what they strategically want to change, and wefind an increasing number of enterprises opting out of managing their own infrastructure, and preferring tomove to AWS to enjoy the agility, innovation, cost-efficiency, and security benefits. And most importantlyfor customers, AWS continues to deliver new capabilities rapidly (over 3,300 new features and
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.5

In [25]:
def similarity_query_byyear(query, year):

    filter={"year": year}

    results_with_scores = db.similarity_search_with_score(query,
      filter=filter)

    print()
    print('Number of Results for the year %s: %s'%(year,len(results_with_scores)))
    print()

    if len(results_with_scores) == 0:
        print('No results')
        exit()
        #print("-" * 80)

    for doc, score in results_with_scores:
        print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

    print("-" * 80)


In [26]:
year=2018
query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)
similarity_query_byyear(query, year)


--------------------------------------------------------------------------------
How has AWS evolved?

Number of Results for the year 2018: 0

No results
--------------------------------------------------------------------------------


In [27]:
year=2019
query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)
similarity_query_byyear(query, year)


--------------------------------------------------------------------------------
How has AWS evolved?

Number of Results for the year 2019: 4

Content: AWS has been successful inincreasing the energy efficiency of its facilities and equipment, for instance by using more efficient evaporativecooling in certain data centers instead of traditional air conditioning. A study by 451 Research found that AWS’sinfrastructure is 3.6 times more energy efficient than the median U.S. enterprise data center surveyed. Along withour use of renewable energy, these factors enable AWS to do the same tasks as traditional data centers with an88% lower carbon footprint. And
Metadata: {'year': 2019, 'source': 'AMZN-2019-Shareholder-Letter.pdf'}
Score: 0.5478576


Content: institutionsaround the world are transitioning from in-person to virtual classrooms and are running on AWS to help ensurecontinuity of learning. And governments are leveraging AWS as a secure platform to build out new capabilitiesin their 

In [28]:
year=2020
query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)
similarity_query_byyear(query, year)


--------------------------------------------------------------------------------
How has AWS evolved?

Number of Results for the year 2020: 4

Content: 100 million smart home devices to Alexa. Amazon Web Services serves millions of customers and ended 2020
with a $50 billion annualized run rate. In 1997, we hadn’t invented Prime, Marketplace, Alexa, or AWS.
They weren’t even ideas then, and none was preordained. We took great risk with each one and put sweat
and ingenuity into each one.
Along the way, we’ve created $1.6 trillion of wealth for shareowners. Who are they? Y our Chair is one, and
Metadata: {'year': 2020, 'source': 'AMZN-2020-Shareholder-Letter.pdf'}
Score: 0.59557676


Content: To our shareowners:
In Amazon’s 1997 letter to shareholders, our first, I talked about our hope to create an “enduring franchise,”
one that would reinvent what it means to serve customers by unlocking the internet’s power. I noted that
Amazon had grown from having 158 employees to 614, and that we 

In [29]:
year=2021
query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)
similarity_query_byyear(query, year)


--------------------------------------------------------------------------------
How has AWS evolved?

Number of Results for the year 2021: 4

Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5201462


Content: from working with colleagues and technology on-premises to working remotely. AWS played amajor role in enabling this business continuity. Whether companies saw extraordinary demand spikes, ordemand diminish quickly with reduced external consumption, the cloud’s elasticity to scale capacity up anddown quickly, as well as AWS’s unusually broad functionality helped millions of companies adjust to thesedifficult circumstances.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5375186


Content: back and determining

In [30]:
year=2022
query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)
similarity_query_byyear(query, year)


--------------------------------------------------------------------------------
How has AWS evolved?

Number of Results for the year 2022: 4

Content: in AWS. Our new customer pipeline is robust, as are our active migrations. Many companies usediscontinuous periods like this to step back and determine what they strategically want to change, and wefind an increasing number of enterprises opting out of managing their own infrastructure, and preferring tomove to AWS to enjoy the agility, innovation, cost-efficiency, and security benefits. And most importantlyfor customers, AWS continues to deliver new capabilities rapidly (over 3,300 new features and
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.5205088


Content: done innovating here,and this long-term investment should prove fruitful for both customers and AWS. AWS is still in the earlystages of its evolution, and has a chance for unusual growth in the next decade.
Metadata: {'year': 2022, 'source': '

# Prompt and Completions Examples

In [31]:
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "who is the President of the USA?"
query2 = "Who won the baseball World Series in 2020? and Who Lost"

device="cuda"
def prompt_completion(query):
    messages = [
        {"role": "user", "content": "%s"%query}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    #https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                    pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids)
    print()
    print()
    result=decoded[0].replace('<s> [INST] %s [/INST]'%query,"")
    result=result.replace('</s>',"")
    print('Prompt: %s'%query)
    print('-'*80)
    print('Answer: %s'%result)

prompt_completion(query)
print()
print('='*80)
prompt_completion(query1)
print()
print('='*80)
prompt_completion(query2)

query3 = "what is the 20.5% of 40?"
query4 = "As a data scientist, can you explain the concept of regularization in machine learning?"
query5 ='Which country has the most natural lakes? Answer with only the country name.'

print()
print('='*80)
prompt_completion(query3)
print()
print('='*80)
prompt_completion(query4)
print()
print('='*80)
prompt_completion(query5)


query6 = "How AWS has evolved?"
print()
print('='*80)
prompt_completion(query6)

query7 = "Summarize this: Mistral LLM and Langchain intgegration. Overview and Tutorial with practical examples. Mistral LLM is a large language model developed by Mistral AI, a French startup making waves in\
the tech community. Mistral LLM is a decoder-based language model with 7 billion parameters, which makes it one of the most significant language models available. \
It uses sliding window attention, grouped query attention, and byte-fallback BPE tokenizer to achieve its impressive performance. \
Mistral LLM and Mistral LLM MoE 8x7B are language models developed by Mistral AI, a French startup. Mistral LLM is a decoder-based language model with 7 billion parameters, which makes it one of the most significant language models available. \
On the other hand, Mistral LLM MoE 8x7B[1c] is an open-weight model that employs a Mixture of Expert (MoE) architecture to generate human-like responses. While there is no direct information about the relationship between Mistral LLM and Mistral LLM MoE 8x7B, it is safe to assume that Mistral LLM MoE 8x7B is an extension of Mistral LLM. \
Mistral LLM MoE 8x7B is a larger model than Mistral LLM, with eight experts, each with seven billion parameters.\
Mistral LLM is designed for various natural language processing tasks, including text generation, summarization, and question-answering.\
It has outperformed other large language models, such as Llama 2 13B, on all benchmarks tested.\
Mistral LLM is available to developers via the gpt-4-vision-preview model and the Chat Completions API, which has been updated to support image inputs. \
The model can be used to understand images and answer questions about them. It is best at answering general questions about what is present in the pictures, but it is not yet optimized to answer detailed questions about the location of specific objects in an image.\
Mistral LLM is a significant step forward in natural language processing. Its impressive performance and large parameter count make it a powerful [3] tool for developers working on a wide range of NLP tasks.\
In summary, Mistral LLM is a powerful language model that has been shown to outperform other large language models on all benchmarks tested.\
It is available to developers via the gpt-4-vision-preview model and the Chat Completions API and can be used for a wide range of natural language processing tasks.\
Mistral LLM has been benchmarked against other large language models, such as Llama 2 13B, and it has been shown to outperform all benchmarks tested.\In particular, Mistral 7B outperforms Llama 2 13B on all metrics measured and is on par with Llama 34B.\
It is important to note that the benchmarks used to compare these models can vary widely, and the results may not be directly comparable.\
However, the fact that Mistral LLM has been shown to outperform other large language models on multiple benchmarks is a testament to its impressive performance.\
LangChain is an open-source AI abstraction library that easily integrates large language models (LLMs) like GPT-4/LLaMa 2 into applications.\
LangChain provides a simplified framework for querying LLMs to generate text, code, translations, and more using Python [9]. LangChain is a generic interface for nearly any LLM, providing a centralized development environment to build and integrate LLM applications with external data sources and software workflows.\LangChain's integration with LLMs like OpenAI, Cohere, and Hugging Face is fundamental to its functionality [11].\
Mistral LLM is a large language model developed by Mistral AI that has been shown to outperform other large language models, such as Llama 2 13B, on all benchmarks tested.\
Mistral LLM is available to developers via the gpt-4-vision-preview model and the Chat Completions API, which has been updated to support image inputs [12].\
The model can be used to understand images and answer questions about them.\
While there is no direct information about the integration of LangChain and Mistral LLM, LangChain's ability to integrate with nearly any LLM suggests that it can be used with Mistral LLM.\
The combination of LangChain and Mistral LLM could provide a powerful tool for developers working on natural language processing tasks [9-14]. However, it is essential to consider the model's limitations as you explore what use cases it can apply to.\
As an illustration of how the concepts and terms mentioned above work together, we implemented a jyputer notebook thoroughly tested in Google Colab based on GPU T4 devices.\
The notebook covers how we can use Langchain using OpenAI embedding and a PostgreSQL extension called pg_embedding to upload actual documents from AWS S3.\
Also, as a model for LLM, we used mistral/Mistral-7B-v0.1 from the hugging face repository, which created an entire open-source integrated ecosystem to support generative AI solutions.\
I also share a notebook of using MLxDL/Mistral_IN_AWS.ipynb at main · frank-morales2020/MLxDL (github.com)\
You can access this case in my GitHub ML/DL development repository shown below: MLxDL/Mistral_Integration_with_Langchain_PostgreSQL.ipynb at main · frank-morales2020/MLxDL (github.com)"

print()
print('='*80)
prompt_completion(query7)




Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------
Answer:  Let's reason through this problem step-by-step:

1. We know that you bought an ice cream for 6 kids, and each cone cost $1.25. So, the total cost of the ice creams is 6 * $1.25 = $7.50.
2. You paid for the ice creams with a $10 bill. Now, we need to determine how much change you received.
3. To do this, subtract the total cost of the ice creams from the amount you paid: $10 - $7.50 = $2.50.

So, you got back $2.50 in change.



Prompt: who is the President of the USA?
--------------------------------------------------------------------------------
Answer:  As of my knowledge up to January 31, 2022, the current President of the United States of America is Joe Biden. However, please verify against real-time information sources for accuracy, as t

In [32]:
device="cuda"
def prompt_completion(query):
    messages = [
        {"role": "user", "content": "%s"%query}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    #https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                    pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids)
    print()
    print()
    result=decoded[0].replace('<s> [INST] %s [/INST]'%query,"")
    result=result.replace('</s>',"")
    print('Prompt: %s'%query)
    print('-'*80)
    print('Answer: %s'%result)

In [33]:
query8= "Summarize this: Introduction In the realm of artificial intelligence (AI[1]) and machine learning (ML), \
three concepts have emerged as significant game-changers: \ Vector Databases[2], Retrieval Augmented Generation (RAG[3]), \
and Semantic Search[4]. These technologies have revolutionized how we store, retrieve, and interpret data, paving the way for more \
intelligent and efficient systems.\Vector Database A Vector Database is a type of database that stores data as high-dimensional vectors[5]. \
These databases provide the ability to store and retrieve vectors as high-dimensional points. They add capabilities for efficient and \
fast-looking nearest neighbours in the N-dimensional space. They are typically powered by k-nearest neighbour (k-NN) indexes and \
built with algorithms like the Hierarchical Navigable Small World (HNSW) and Inverted File Index (IVF) algorithms. Vector databases \
are typically used to power vector search cases like visual, semantic, and multimodal search. In computer science, an Inverted Index, \
also referred to as a postings list, postings file, or inverted file, is a database index that stores a mapping from content, such as \
words or numbers, to its locations in a table[6], document or a set of documents. This contrasts with a forward index, which maps from \
documents to content. An inverted index allows fast full-text searches at increased processing costs when a document is added to the database. \
It is the most popular data structure used in document retrieval systems, used on a large scale, for example, in search engines[7].\
There are two main variants of inverted indexes: A record-level inverted index contains a list of references to documents for each word[8]. \
A word-level inverted index also contains each word’s position within a document. The latter form offers more functionality (like phrase searches) \
but needs more processing power and space to be created[8]. The inverted index data structure is a central component of a typical search engine \
indexing algorithm. A goal of a search engine implementation is to optimize the speed of the query: find the documents where word X occurs. \
Once a forward index is developed, which stores lists of words per document, it is next inverted[9] to create an inverted index. Querying \
the forward index would require sequential iteration through each document and to each word to verify a matching document. The time, memory, \
and processing resources to perform such a query are only sometimes technically realistic. Instead of listing the words per document in the forward \
index, the inverted index data structure, which lists the documents per word, is developed. With the inverted index created, the query can be resolved \
by jumping to the word ID (via random access) in the inverted index. In bioinformatics, inverted indexes are crucial in the sequence assembly of short \
fragments of sequenced DNA. One way to find the source of a fragment is to search for it against a reference DNA sequence. \
A small number of mismatches (due to differences between the sequenced DNA and reference DNA, or errors) can be accounted for by dividing the fragment\
into smaller fragments — at least one subfragment is likely to match the reference DNA sequence. \
The matching requires constructing an inverted index of all substrings of a certain length from the reference DNA sequence. Retrieval Augmented Generation (RAG)\
Retrieval Augmented Generation (RAG) is a pattern that works with pre-trained Large Language Models (LLM) and your data to generate responses. \
RAG is an AI framework for retrieving facts from an external knowledge base to ground large language models (LLMs) on the most accurate, \
up-to-date information and to give users insight into LLMs’ generative process. It ensures that the model has access to the most current, \
reliable facts and that users have access to its sources, ensuring that its claims can be checked for accuracy and ultimately trusted[10][11].\
RAG is an AI framework that augments the capabilities of a Large Language Model (LLM) like GPT-4 by adding an information retrieval system \
that provides grounding data. This means you can constrain generative AI to your enterprise content sourced from vectorized documents, images, audio, \
and video[5]. Semantic Search Semantic search is a search engine technology that interprets the meaning of words and phrases[12]. \
It seeks to improve search accuracy by understanding the searcher’s intent and the contextual meaning of terms as they appear in the searchable[13] \
database. The results of a semantic search will return content matching the meaning of a query instead of content that matches words in the query. \
Applications with Test Cases As an illustration of how to implement those three concepts, we are using some modified notebooks that I adapted to be \
run with Google Colab. Google Colab, or “Collaboratory,” is a free online service allowing you to run Python code in your browser with access to GPUs \
and TPUs. RAG is a pattern that generates responses with pre-trained LLM and your data. RAG is an AI framework for retrieving facts from an external \
knowledge base to ground large language models (LLMs) on the most accurate, up-to-date information and to give users insight into LLMs’ generative \
process. It ensures that the model has access to the most current, reliable facts and that users have access to its sources, ensuring that its \
claims can be checked for accuracy and ultimately trusted. In AI’s context, Vector Databases and RAG play crucial roles. Vector Databases efficiently \
store and retrieve high-dimensional data, essential for many AI applications. On the other hand, RAG improves the quality of responses generated \
by Large Language Models by grounding the model on external sources of knowledge[9]. Two examples of using both RAG with vector DataBase and OpenAI API \
embeddings are in the notebooks below, which I modified based on OpenAI models gpt-3.5-turbo-0613 (GPT-3.5) and gpt-4–0613(GPT-4), respectively. \
Notebook with OpenAI MODEL GPT-3.5 Notebook with OpenAI MODEL GPT-4 The Notebooks were tested successfully in the Google Cloud using Google Colab. \
The audience will also learn how to use and install PostgreSQL with the pgvector extension in Google Colab and securely use the OpenAI KEY \
in the Google Cloud. The pgvector extension is a powerful tool that turns PostgreSQL into a vector database. It introduces a dedicated data type, \
operators, and functions that enable efficient storage, manipulation, and analysis of vector data directly within the PostgreSQL database[14]. \
Here are some key features and use cases: Vector storage: The pgvector extension lets you store high-dimensional vectors in PostgreSQL tables. \
It provides a dedicated data type for vector representation, allowing efficient storage and retrieval of vector data. Similarity search: With pgvector, \
you can perform similarity searches based on vector similarity metrics such as cosine similarity or Euclidean distance. \
Natural Language Processing (NLP) and text analysis: pgvector is particularly useful in NLP applications. It allows you to represent \
text documents as vectors using word embeddings or document embeddings. Computer vision: The pgvector extension can handle vector \
representations of images and enable similarity-based image search. Conclusion In conclusion, Vector Databases, Retrieval Augmented Generation, \
and Semantic Search are three powerful technologies shaping AI and ML’s future. By enabling more efficient data storage and retrieval, \
grounding AI models in reliable facts, and interpreting search queries more meaningfully, these technologies pave the way for more intelligent and \
efficient systems. We expect to see more innovative applications and use cases emerge as these technologies evolve."

print()
print('='*80)
prompt_completion(query8)




Prompt: Summarize this: Introduction In the realm of artificial intelligence (AI[1]) and machine learning (ML), three concepts have emerged as significant game-changers: \ Vector Databases[2], Retrieval Augmented Generation (RAG[3]), and Semantic Search[4]. These technologies have revolutionized how we store, retrieve, and interpret data, paving the way for more intelligent and efficient systems.\Vector Database A Vector Database is a type of database that stores data as high-dimensional vectors[5]. These databases provide the ability to store and retrieve vectors as high-dimensional points. They add capabilities for efficient and fast-looking nearest neighbours in the N-dimensional space. They are typically powered by k-nearest neighbour (k-NN) indexes and built with algorithms like the Hierarchical Navigable Small World (HNSW) and Inverted File Index (IVF) algorithms. Vector databases are typically used to power vector search cases like visual, semantic, and multimodal search. In 

In [34]:
import colab_env
import os

access_token = os.getenv("HF_TOKEN")
device = "cuda" # the device to load the model onto

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig, GenerationConfig, pipeline
from transformers import AutoTokenizer, MistralForCausalLM

#model = MistralForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",padding_side="left")
#tokenizer.pad_token = tokenizer.eos_token # Most LLMs don't have a pad token by default

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py

# Generate
#print()
#generate_ids = model.generate(inputs.input_ids, max_length=30)

#generate_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
#                pad_token_id=tokenizer.eos_token_id)

#response=tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#print(response)
print()

#query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query="What is your favourite condiment?"
messages = [
    {"role": "user", "content": "%s"%query},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
#model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded[0])


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!  [INST] Do you have mayonnaise recipes? [/INST] Yes, I'd be happy to share a simple mayonnaise recipe with you! Here it is:

Ingredients:

* 2 large egg yolks
* 1 tablespoon Dijon mustard
* 1 tablespoon white vinegar
* 1/4 teaspoon salt
* 2/3 cup vegetable oil
* 1/4 cup olive oil
* Salt and pepper to taste

Instructions:
1. In a small bowl, whisk together the egg yolks, Dijon mustard, white vinegar, and salt.
2. Slowly stream in the vegetable oil while whisking, starting with a few drops at a time to prevent the egg mixture from curdling.
3. Once the vegetable oil is fully incorporated, whisk in the olive oil in a gentle stream.
4. Season with salt and pepper to taste.
5. Refrigerate immediately until ready to serve.

And that's it! Mayonnaise is a great condiment to have on hand, and i

In [35]:
#!pip install transformers --upgrade
#device = "cuda"
from IPython.display import Markdown, display

query = "who is Barack Obama?"
result = llm(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))
print()
print("-" * 80)
print()

<b>who is Barack Obama?</b>

<p>
A: a black American politician and the 44th President of the United States.</p>


--------------------------------------------------------------------------------



In [36]:
query = "who is Steve Jobs?"
result = llm(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))
print()
print("-" * 80)
print()

<b>who is Steve Jobs?</b>

<p>
User 1: Founder and CEO of Apple Inc.

Born in 1957, Steve Jobs was a pivotal figure in the development of personal computing and the popularization of graphical user interfaces. He co-founded Apple Inc., one of the most successful technology companies in history, with his friend Steve Wozniak. He led the company through its early years and played an instrumental role in shaping the Macintosh computer, which revolutionized personal computing by featuring a mouse, graphical interface, and built-in networking capabilities. After being fired from Apple in 1985 due to disagreements over product direction, Jobs went on to found NeXT Computer, which eventually merged with Apple in 1997. During this period, he also founded Pixar Animation Studios, which developed groundbreaking computer animated films such as "Toy Story" and "Finding Nemo." Jobs returned to Apple in 2001 and spearheaded the revival of the company under his leadership.</p>


--------------------------------------------------------------------------------



In [37]:
from langchain.llms import OpenAI
#import colab_env
from IPython.display import Markdown, display

from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

#retriever = db.as_retriever(search_type="similarity_score_threshold",search_kwargs='1')
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create a chain to answer questions
#qa = RetrievalQA.from_chain_type(
#     llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

qa = RetrievalQA.from_chain_type(
     llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "How AWS has evolved?"

result = qa(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

print()
print("-" * 80)
print()

print()
print('CHAIN to answer questions')
print("-" * 80)
result = qa({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

<b>How AWS has evolved?</b>

<p>{'query': 'How AWS has evolved?', 'result': '', 'source_documents': [Document(page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.', metadata={'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}), Document(page_content='We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage their technology infrastructure. Amazon would be a different company ifwe’d slowed investment in AWS during that 2008-2009', metadata={'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'})]}</p>


--------------------------------------------------------------------------------


CHAIN to answer questions
--------------------------------------------------------------------------------
Query: How AWS has evolved?

Result:  AWS has evolved significantly since its initial launch and has become a major player in the cloud computing industry, with an $85B annual revenue run rate business and strong profitability. The long-term decision to continue investing in AWS has played a crucial role in its growth and transformation over the past fifteen years.

Context Documents: 
page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.' metadata={'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}

page_content='We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made 